In [ ]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go

In [ ]:
with open('../data/nash_hosp_referrers.pickle', 'rb') as file:
    nash_hosp_referrers = pickle.load(file)

In [ ]:
with open('../data/nash_hosp_referrers_b.pickle', 'rb') as file:
    nash_hosp_referrers_b = pickle.load(file)

In [ ]:
# Combine different names associated with the same broader hospital system, shorten names for visuals
nash_hosp_referrers['hospital'] = (
    nash_hosp_referrers['hospital']
    .str.replace('SETON CORPORATION', 'SAINT THOMAS')
    .str.replace('SAINT THOMAS WEST HOSPITAL', 'SAINT THOMAS')
    .str.replace('HCA HEALTH SERVICES OF TENNESSEE, INC.', 'HCA/TRISTAR')
    .str.replace('HTI MEMORIAL HOSPITAL CORPORATION', 'HCA/TRISTAR')
    .str.replace('NASHVILLE GENERAL HOSPITAL', 'NASHVILLE GENERAL')
    .str.replace('VANDERBILT UNIVERSITY MEDICAL CENTER', 'VUMC')
)

In [ ]:
nash_hosp_referrers_b['hospital'] = (
    nash_hosp_referrers_b['hospital']
    .str.replace('SETON CORPORATION', 'SAINT THOMAS')
    .str.replace('SAINT THOMAS WEST HOSPITAL', 'SAINT THOMAS')
    .str.replace('HCA HEALTH SERVICES OF TENNESSEE, INC.', 'HCA/TRISTAR')
    .str.replace('HTI MEMORIAL HOSPITAL CORPORATION', 'HCA/TRISTAR')
    .str.replace('NASHVILLE GENERAL HOSPITAL', 'NASHVILLE GENERAL')
    .str.replace('VANDERBILT UNIVERSITY MEDICAL CENTER', 'VUMC')
)

In [ ]:
#add a column that calculates the total wait in days for each providers' referrals
nash_hosp_referrers_b['total_wait'] = (nash_hosp_referrers_b['transaction_count'] 
                                       * 
                                       nash_hosp_referrers_b['average_day_wait'])

nash_hosp_referrers_b

In [ ]:
# Create a table that aggregates referral information by hospital
hosp_summary_values_b = (
    nash_hosp_referrers_b
    .groupby('hospital')
    .agg(
        total_providers = ('from_npi', 'count'),
        total_transactions = ('transaction_count', 'sum'),
        total_patients = ('patient_count', 'sum'),
        total_states = ('location_address_state_name', 'nunique'),
        total_specialties = ('specialty', 'nunique'),
        total_wait = ('total_wait', 'sum')
    )
    .reset_index()
)

hosp_summary_values_b['avg_day_wait'] = round(hosp_summary_values_b['total_wait'] 
                                              / 
                                              hosp_summary_values_b['total_transactions'],
                                              2)

hosp_summary_values_b

In [ ]:
# Plot of total referral transactions by hospital
df = hosp_summary_values_b.sort_values('total_transactions')

fig = px.bar(df,
             x='hospital', 
             y='total_transactions',
             color='hospital',
             labels = {'total_transactions' : 'Total Referral Transactions',
                       'hospital' : 'Hospital'},
             color_discrete_map={
                 "HCA/TRISTAR":"#AA1E2D",
                 "NASHVILLE GENERAL":"#FDE74C",
                 "SAINT THOMAS":"#8FB8DE",
                 "VUMC":"#9A94BC"
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 tickangle = 45)

fig.update_layout(showlegend=False)

fig.update_layout(title_text="Referral Transactions by Hospital",
                    title_xanchor="auto",
                    title_font_size=18,
                    yaxis={'categoryorder':'category descending'}
                   )

fig.show()

In [ ]:
#fig.write_html("../data/visuals/hosp_transactions.html")

In [ ]:
# Plot of total referring providers by hospital
df = hosp_summary_values_b.sort_values('total_providers')

fig = px.bar(df,
             x='hospital', 
             y='total_providers',
             color='hospital',
             labels = {'total_providers' : 'Total Referring Providers',
                       'hospital' : 'Hospital'},
             color_discrete_map={
                 "HCA/TRISTAR":"#AA1E2D",
                 "NASHVILLE GENERAL":"#FDE74C",
                 "SAINT THOMAS":"#8FB8DE",
                 "VUMC":"#9A94BC"
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 tickangle = 45)

fig.update_layout(showlegend=False)

fig.update_layout(title_text="Referring Providers by Hospital",
                    title_xanchor="auto",
                    title_font_size=18,
                    yaxis={'categoryorder':'category descending'}
                   )


fig.show()

In [ ]:
#fig.write_html("../data/visuals/hosp_providers.html")

In [ ]:
# Plot of total referred patients by hospital
df = hosp_summary_values_b.sort_values('total_patients')

fig = px.bar(df,
             x='hospital', 
             y='total_patients',
             color='hospital',
             labels = {'total_patients' : 'Total Patients Referred',
                       'hospital' : 'Hospital'},
             color_discrete_map={
                 "HCA/TRISTAR":"#AA1E2D",
                 "NASHVILLE GENERAL":"#FDE74C",
                 "SAINT THOMAS":"#8FB8DE",
                 "VUMC":"#9A94BC"
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 tickangle = 45)

fig.update_layout(showlegend=False)

fig.update_layout(title_text="Referred Patients by Hospital",
                    title_xanchor="auto",
                    title_font_size=18,
                    yaxis={'categoryorder':'category descending'}
                   )

fig.show()

In [ ]:
#fig.write_html("../data/visuals/hosp_patients.html")

In [ ]:
# Plot of total overall average wait time by hospital
df = hosp_summary_values_b.sort_values('avg_day_wait')

fig = px.bar(df,
             x='hospital', 
             y='avg_day_wait',
             color='hospital',
             labels = {'avg_day_wait' : 'Average Wait (days)',
                       'hospital' : 'Hospital'},
             color_discrete_map={
                 "HCA/TRISTAR":"#AA1E2D",
                 "NASHVILLE GENERAL":"#FDE74C",
                 "SAINT THOMAS":"#8FB8DE",
                 "VUMC":"#9A94BC"
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 tickangle = 45)

fig.update_layout(showlegend=False)

fig.update_layout(title_text="Average Wait by Hospital",
                    title_xanchor="auto",
                    title_font_size=18,
                    yaxis={'categoryorder':'category descending'}
                   )

fig.show()

In [ ]:
#fig.write_html("../data/visuals/hosp_wait.html")

In [ ]:
# Aggregating by provider and hospital
top_hosp_ref = (nash_hosp_referrers_b
                .groupby(['from_npi',
                         'first_name',
                         'last_name',
                         'credential',
                         'location_address_state_name', 
                         'location_address_city_name', 
                         'specialty',
                         'sub_specialty',
                         'hospital'])
                .agg(total_transactions = ('transaction_count', 'sum'),
                     total_patients = ('patient_count', 'sum'),
                     total_wait = ('total_wait', 'sum')
                    )
                .reset_index())

top_hosp_ref

In [ ]:

top_hosp_ref['hosp_top_referrals'] = (top_hosp_ref
                                      .groupby('hospital')['total_patients']
                                      .rank(ascending = False)
)

top_hosp_ref

In [ ]:
top_hosp_ref['from_npi'] = top_hosp_ref['from_npi'].astype(str)

In [ ]:
top_hosp_ref.loc[top_hosp_ref['hosp_top_referrals'] <= 5].sort_values(by= ['hospital', 'hosp_top_referrals'])


In [ ]:
df = (top_hosp_ref
      .loc[top_hosp_ref['hosp_top_referrals'] <= 5]
      .sort_values(by= ['hospital', 'hosp_top_referrals'])
     )

fig = px.bar(df,
             x='from_npi', 
             y='total_patients',
             color='hospital',
             labels = {'total_patients' : 'Total Patients Referred',
                       'hospital' : 'Hospital',
                       'from_npi' : 'Provider NPI',
                       'first_name' : 'Provider First Name',
                       'last_name' : 'Provider Last Name',
                       'credential' : 'Credential',
                       'sub_specialty' : 'Sub-Specialty',
                       'total_transactions' : 'Total Referral Transactions'},
             color_discrete_map={
                 "HCA/TRISTAR":"#AA1E2D",
                 "NASHVILLE GENERAL":"#FDE74C",
                 "SAINT THOMAS":"#8FB8DE",
                 "VUMC":"#9A94BC"
             },
             hover_name = 'hospital',
             hover_data={
                 'hospital' : False,
                 'first_name' : True,
                 'last_name' : True,
                 'credential' : True,
                 'sub_specialty' : True,
                 'total_transactions' : True
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 showticklabels=False)


fig.show()

In [ ]:
serv_specialties = ['Anatomic Pathology & Clinical Pathology Physician', 
                    'Diagnostic Radiology Physician', 
                    'Emergency Medicine Physician']

In [ ]:
top_hosp_ref.loc[top_hosp_ref['sub_specialty'].isin(serv_specialties)]

In [ ]:
# Do a version of the above visual without service-based specialties
top_hosp_ref['hosp_top_referrals_non_serv'] = (top_hosp_ref
                                      .loc[~top_hosp_ref['sub_specialty'].isin(serv_specialties)]
                                      .groupby('hospital')['total_patients']
                                      .rank(ascending = False)
)

top_hosp_ref.loc[top_hosp_ref['hosp_top_referrals_non_serv'].isin([1, 2, 3, 4, 5])]

In [ ]:
df = (top_hosp_ref
      .loc[top_hosp_ref['hosp_top_referrals_non_serv'].isin([1, 2, 3, 4, 5])]
      .sort_values(by= ['hospital', 'hosp_top_referrals_non_serv'])
     )

fig = px.bar(df,
             x='from_npi', 
             y='total_patients',
             color='hospital',
             labels = {'total_patients' : 'Total Patients Referred',
                       'hospital' : 'Hospital',
                       'from_npi' : 'Provider NPI',
                       'first_name' : 'Provider First Name',
                       'last_name' : 'Provider Last Name',
                       'credential' : 'Credential',
                       'sub_specialty' : 'Sub-Specialty',
                       'total_transactions' : 'Total Referral Transactions'},
             color_discrete_map={
                 "HCA/TRISTAR":"#AA1E2D",
                 "NASHVILLE GENERAL":"#FDE74C",
                 "SAINT THOMAS":"#8FB8DE",
                 "VUMC":"#9A94BC"
             },
             hover_name = 'hospital',
             hover_data={
                 'hospital' : False,
                 'first_name' : True,
                 'last_name' : True,
                 'credential' : True,
                 'sub_specialty' : True,
                 'total_transactions' : True
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 showticklabels=False)


fig.show()

In [ ]:
#fig.write_html("../data/visuals/hosp_top_ref_non_serv.html")

In [ ]:
vandy_ref = nash_hosp_referrers_b.loc[nash_hosp_referrers_b['hospital'].str.contains("VUMC")]['from_npi'].unique().tolist()
vandy_ref

In [ ]:
(nash_hosp_referrers_b
    .loc[(~nash_hosp_referrers_b['from_npi'].isin(vandy_ref))
         &
         (nash_hosp_referrers_b['location_address_state_name'] == 'TN')]
    .groupby(['from_npi',
                 'first_name',
                 'last_name',
                 'credential',
                 'location_address_state_name', 
                 'location_address_city_name', 
                 'specialty',
                 'sub_specialty'])
    .agg(total_patients = ('patient_count', 'sum'),
         total_transactions = ('transaction_count', 'sum'))
    .reset_index()
    .sort_values('total_patients', ascending = False)
    .head(25)
)    

In [ ]:
df = (
    nash_hosp_referrers_b
    .loc[(~nash_hosp_referrers_b['from_npi'].isin(vandy_ref))
         &
         (nash_hosp_referrers_b['location_address_state_name'] == 'TN')]
    .groupby(['from_npi',
                 'first_name',
                 'last_name',
                 'credential',
                 'location_address_state_name', 
                 'location_address_city_name', 
                 'specialty',
                 'sub_specialty'])
    .agg(total_patients = ('patient_count', 'sum'),
         total_transactions = ('transaction_count', 'sum'))
    .reset_index()
    .sort_values('total_patients', ascending = False)
    .head(25)
)


df['from_npi'] = df['from_npi'].astype(str)
fig = px.bar(df,
             x='from_npi', 
             y='total_patients',
             color='total_patients',
             color_continuous_scale='Darkmint',
             labels = {'total_patients' : 'Total Patients Referred',
                       'from_npi' : 'Provider NPI',
                       'first_name' : 'Provider First Name',
                       'last_name' : 'Provider Last Name',
                       'credential' : 'Credential',
                       'sub_specialty' : 'Sub-Specialty',
                       'total_transactions' : 'Total Referral Transactions'},
             hover_data={
                 'first_name' : True,
                 'last_name' : True,
                 'credential' : True,
                 'sub_specialty' : True,
                 'total_transactions' : True
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 showticklabels=False)


fig.show()

In [ ]:
fig.write_html("../data/visuals/individual_no_vandy.html")

In [ ]:
(
    nash_hosp_referrers_b
    .loc[(~nash_hosp_referrers_b['from_npi'].isin(vandy_ref))
         &
         (nash_hosp_referrers_b['location_address_state_name'] == 'TN')
         &
         (~nash_hosp_referrers_b['sub_specialty'].isin(serv_specialties))]
    .groupby(['from_npi',
                 'first_name',
                 'last_name',
                 'credential',
                 'location_address_state_name', 
                 'location_address_city_name', 
                 'specialty',
                 'sub_specialty',
             'hospital'])
    .agg(total_patients = ('patient_count', 'sum'),
         total_transactions = ('transaction_count', 'sum'))
    .reset_index()
    .sort_values('total_patients', ascending = False)
    .head(25)
)


In [ ]:
df = (
    nash_hosp_referrers_b
    .loc[(~nash_hosp_referrers_b['from_npi'].isin(vandy_ref))
         &
         (nash_hosp_referrers_b['location_address_state_name'] == 'TN')
         &
         (~nash_hosp_referrers_b['sub_specialty'].isin(serv_specialties))]
    .groupby(['from_npi',
                 'first_name',
                 'last_name',
                 'credential',
                 'location_address_state_name', 
                 'location_address_city_name', 
                 'specialty',
                 'sub_specialty'])
    .agg(total_patients = ('patient_count', 'sum'),
         total_transactions = ('transaction_count', 'sum'))
    .reset_index()
    .sort_values('total_patients', ascending = False)
    .head(25)
)


df['from_npi'] = df['from_npi'].astype(str)
fig = px.bar(df,
             x='from_npi', 
             y='total_patients',
             color='total_patients',
             color_continuous_scale='Darkmint',
             labels = {'total_patients' : 'Total Patients Referred',
                       'from_npi' : 'Provider NPI',
                       'first_name' : 'Provider First Name',
                       'last_name' : 'Provider Last Name',
                       'credential' : 'Credential',
                       'sub_specialty' : 'Sub-Specialty',
                       'total_transactions' : 'Total Referral Transactions'},
             hover_data={
                 'first_name' : True,
                 'last_name' : True,
                 'credential' : True,
                 'sub_specialty' : True,
                 'total_transactions' : True
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 showticklabels=False)


fig.show()

In [ ]:
#fig.write_html("../data/visuals/individual_no_vandy_no_serv.html")

In [ ]:
df = (
    nash_hosp_referrers_b
    .loc[(~nash_hosp_referrers_b['from_npi'].isin(vandy_ref))
         &
         (nash_hosp_referrers_b['location_address_state_name'] == 'TN')
         &
         (~nash_hosp_referrers_b['sub_specialty'].isin(serv_specialties))]
    .groupby('sub_specialty')
    .agg(total_patients = ('patient_count', 'sum'),
         total_transactions = ('transaction_count', 'sum'))
    .reset_index()
    .sort_values('total_patients', ascending = False)
    .head(25)
)


fig = px.bar(df,
             x='sub_specialty', 
             y='total_patients',
             color='total_patients',
             color_continuous_scale='Bluyl',
             labels = {'total_patients' : 'Total Patients',
                       'sub_specialty' : 'Sub-Specialty',
                       'total_transactions' : 'Total Referral Transactions'},
             hover_data={
                 'sub_specialty' : True,
                 'total_transactions' : True
             },
             template="simple_white")

fig.update_xaxes(title=None,
                 showticklabels=False)


fig.show()

In [ ]:
#fig.write_html("../data/visuals/specialty_no_vandy_no_serv.html")